In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ImageManipulation as IM
import os
import findPeaksLib
import GaussFits
import pickle
from GaussAnalysisPipeline import *
import precisionFuncs as pF

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec  9 10:35:29 2019

@author: buddeja
"""
import shutil
import pandas as pd

def mergePTUfiles(path, savepath):
    files = os.listdir(path)
    file_folder = []


    if os.path.exists(savepath) == True:
         print('savepath_exists')
    else:
        os.makedirs(savepath)  

    for i in files:
        if i.startswith('Overview'):
            file_folder.append(i)

    for ii in file_folder:
        print(ii)

        files_ptu = []
        files_xcoord = []
        files_ycoord = []
        x_relative = []
        y_relative = []
        folderpath = '{}{}{}'.format(path, ii, '/')
        files_folderpath = os.listdir(folderpath)
        for i in files_folderpath:
            if i.endswith('.ptu'):
                files_ptu.append(i)
            elif i.endswith('x_transfer.dat.dat'):
                x_coord = i
            elif i.endswith('y_transfer.dat.dat'):
                y_coord = i
        if len(files_ptu) >0 :

            filex = pd.read_csv('{}{}'.format(folderpath,x_coord),sep='\t')
            filey = pd.read_csv('{}{}'.format(folderpath,y_coord),sep='\t')
            filex = [i[0] for i in filex.values.tolist()]
            filey = [i[0] for i in filey.values.tolist()]
            x_relative.extend(filex) 
            y_relative.extend(filey) 
            for i in range(len(files_ptu)):
                name_new = '{}{}{}{}{}{}'.format(files_ptu[i][:-4],'_x_',x_relative[i],'_y_',y_relative[i],'.ptu')
                shutil.copy('{}{}'.format(folderpath, files_ptu[i]), '{}{}'.format(savepath, name_new))

In [4]:
def plotSinglePair(locLst, pxSize = 10):
    coords = np.zeros([len(locLst),2])
    for i, loc in enumerate(locLst):
        coords[i] = loc['G'].spotLst[0].coord - loc['Y'].spotLst[0].coord
    coords *= pxSize
    coords = kickvector(coords, 50)
    coords = coords - np.mean(coords, axis = 0)
    plt.plot(coords[:,0], coords[:,1], '.')
    plt.plot([0,0], [-100,+100], 'r--')
    plt.plot([-100,+100], [0,0], 'r--')
    ax = plt.gca()
    ax.set_aspect('equal')
    plt.xlim([-50, 50])
    plt.ylim([-50, 50])
    stdx, stdy = np.std(coords, axis = 0)
    print('standard deviation of spots is %.2f and %.2f' % (stdx, stdy))
    plt.show()
    return coords

def kickvector(dat, maxval):
    return dat[np.linalg.norm(dat, axis = 1) < maxval]

def plotOccurence(locLst):
    """plots a 2D histogram of how many spots have been fitted
    in the Green and red channel.
    Takes as argument a spotLst Lst"""
    occurence = np.zeros([4,4], np.int)
    for loc in locLst:
        Gspots = len(loc['G'].spotLst)
        Yspots = len(loc['Y'].spotLst)
        occurence[Gspots, Yspots] += 1
    plt.figure(figsize = [4,4])
    x, y = np.meshgrid(np.arange(4),np.arange(4))
    plt.scatter(x,y, s = occurence)
    ax = plt.gca()
    ax.set_xticks([0,1,2,3])
    ax.set_yticks([0,1,2,3])
    plt.ylabel ('# Green spots identified')
    plt.xlabel ('# red spots identified')
    plt.show
    return occurence
    

    
def selectSpotOccurence(locLst, Gspots, Yspots):
    """selects from locLst those localisations that have
    Gspots and Yspots. Gspots and Spots must be lists.
    returns: cleaned locLst"""
    i=0
    locLst_copy = locLst.copy()
    while i < len(locLst_copy):
        if (len(locLst_copy[i]['G'].spotLst) in Gspots and
                len(locLst_copy[i]['Y'].spotLst) in Yspots):
            i += 1
        else:
            locLst_copy.pop(i)
    return locLst_copy

In [5]:
def analyseSTEDdir(locLst, options, wdir, files, Ggate = 0, Ygate = 0, 
                   DTwoIstar = 0.03, garbageBrightness = 20, junkIstar = 0.30,
                   outname = ''):
    ntacs = 256
    ROIsize = 30

    for i, file in enumerate(files):
        if file[-4:] != '.ptu':
            print('not a .ptu file, skipping')
            continue
        print('analysing image no. %i' %i)
        ffile = os.path.join(wdir, file)

        #load image, gate
        CLR = IM.processLifetimeImage(ffile.encode(), uselines = np.array([1,2]), ntacs = ntacs)
        CLR.loadLifetime()
        CLR.gate(Ggate, 150, channel = 'G')
        CLR.gate(Ygate, 150, channel = 'Y')
        CLR.loadIntensity()

        #get ROI based on Y signal
        ROI = getROI(CLR.workIntensity.Y, ROIsize)    
        
        #check that ROI is not touching borders:
        try:
            crop(CLR.workIntensity.G, ROI)
        except IndexError:
            print('ROI touches image borders, skipping')
            continue
            
        locLst.append({})
        locLst[-1]['filepath'] = ffile
        #loop over G and Y channels
        for color, bitmap in CLR.workIntensity.__dict__.items():
            if color in ['G', 'Y']:
                outdir = ffile[:-4] + '_' + color

                ROIsnip = crop(bitmap, ROI)
                #fits 1, 2 or 3 gauss spots and determines which one is best
                #returns optimised parameter array
                bestfit, twoIstar, _ = fitNGauss (ROIsnip, options, 
                                                  DTwoIstar = DTwoIstar, garbageBrightness = garbageBrightness,
                                                  junkIstar = junkIstar,
                                                  verbose = False, outdir = outdir)

                #build array containing all the fit results
                locLst[-1][color] = Channel(bitmap)
                locLst[-1][color].fillSpotLst(bestfit)
    
    fpath = os.path.join(wdir, outname)
    
    with open(fpath, 'wb') as output:
        pickle.dump(locLst, output, 1)
        
    return locLst

# Analyzing data sets new

### November 18 2019

In [6]:
wdir = r'N:\Singlem\singlem19-4\November\18_CLR_rois_NV\all'
files = os.listdir(wdir)[:10]
options = optionsCluster(fitbg = 0, setbg = 0.2, ellipt_circ = 1)
CLR18Nov = []
analyseSTEDdir(CLR18Nov, options, wdir, files, 
               Ggate = 29, Ygate = 29, 
               DTwoIstar = 0.03, garbageBrightness = 20, junkIstar = 0.40,
               outname = 'CLR18Nov.spots')

analysing image no. 0
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-37-22.txt'
number of records is 451296


K:\vanderVoortN\FRC\Code\GaussAnalysisPipeline.py:319: RuntimeWarning: divide by zero encountered in true_divide
  plt.imshow(model - image *( 1 + np.log(model / image)), cmap = 'hot')
K:\vanderVoortN\FRC\Code\GaussAnalysisPipeline.py:319: RuntimeWarning: invalid value encountered in multiply
  plt.imshow(model - image *( 1 + np.log(model / image)), cmap = 'hot')


analysing image no. 1
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-37-49.txt'
number of records is 440476
analysing image no. 2
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-38-16.txt'
number of records is 437328


K:\vanderVoortN\FRC\Code\GaussAnalysisPipeline.py:297: UserWarning: No contour levels were found within the data range.
  plt.contour(model, levels = np.array([0.01, 0.5, 1, 3, 5, 10, 20, 30, 40], dtype = np.double))
K:\vanderVoortN\FRC\Code\GaussAnalysisPipeline.py:319: RuntimeWarning: invalid value encountered in log
  plt.imshow(model - image *( 1 + np.log(model / image)), cmap = 'hot')
K:\vanderVoortN\FRC\Code\GaussAnalysisPipeline.py:141: RuntimeWarning: invalid value encountered in greater
  isNoJunkIstar = twoIstar > junkIstar
K:\vanderVoortN\FRC\Code\GaussAnalysisPipeline.py:146: RuntimeWarning: invalid value encountered in less
  isSignificantlyLower[i] = (twoIstar[i] + DTwoIstar < twoIstar[: i]).all()


analysing image no. 3
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-38-43.txt'
number of records is 442560
analysing image no. 4
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-39-10.txt'
number of records is 430556
analysing image no. 5
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-39-37.txt'
number of records is 437831
analysing image no. 6
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-40-04.txt'
number of records is 433523
analysing image no. 7
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_2019-11-18_13-40-31.txt'
number of records is 442718
analysing image no. 8
header dir already exists
b'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\header\\PQSpcm_201

[{'filepath': 'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\PQSpcm_2019-11-18_13-37-22.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c98814c8>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c9b47908>},
 {'filepath': 'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\PQSpcm_2019-11-18_13-37-49.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c9c5e6c8>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c98a2f88>},
 {'filepath': 'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\PQSpcm_2019-11-18_13-38-16.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c9af6448>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c9f42248>},
 {'filepath': 'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\PQSpcm_2019-11-18_13-38-43.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c8670d08>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c99fe948>},
 {'filepath': 'N:\\Singlem\\singlem19-4\\November\\18_CLR_rois_NV\\all\\PQSpcm_2019-11-18_13-39-10.ptu',
  'G': <GaussAn

<Figure size 432x288 with 0 Axes>

### Januari 16, 2020

In [35]:
path = os.path.join(r'N:\Singlem\singlem20-1\January\16_CLR_overnight\run1', '')
savedir = os.path.join(path, 'all')
mergePTUfiles(path, savedir)

Overview_-0.00285404_numberSPOTS_0
Overview_-0.00287604_numberSPOTS_1
Overview_-0.00289804_numberSPOTS_2
Overview_-0.00292004_numberSPOTS_4
Overview_-0.00294204_numberSPOTS_3
Overview_-0.00296404_numberSPOTS_40
Overview_-0.0029860399999999997_numberSPOTS_14
Overview_-0.00300804_numberSPOTS_19
Overview_-0.00303004_numberSPOTS_34
Overview_-0.00305204_numberSPOTS_36
Overview_-0.00307404_numberSPOTS_11
Overview_-0.00309604_numberSPOTS_26
Overview_-0.00311804_numberSPOTS_12
Overview_-0.00314004_numberSPOTS_10
Overview_-0.00316204_numberSPOTS_40
Overview_-0.00318404_numberSPOTS_20
Overview_-0.00320604_numberSPOTS_20
Overview_-0.0032280399999999997_numberSPOTS_13
Overview_-0.00325004_numberSPOTS_33
Overview_-0.00327204_numberSPOTS_15
Overview_-0.00329404_numberSPOTS_27
Overview_-0.00331604_numberSPOTS_34
Overview_-0.00333804_numberSPOTS_20
Overview_-0.00336004_numberSPOTS_9
Overview_-0.0033820400000000002_numberSPOTS_10
Overview_-0.00340404_numberSPOTS_12
Overview_-0.00342604_numberSPOTS_24
O

NotADirectoryError: [WinError 267] The directory name is invalid: 'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\Overview_image.tiff/'

In [7]:
wdir = r'N:\Singlem\singlem20-1\January\16_CLR_overnight\run1\all'
files = os.listdir(wdir)
options = optionsCluster(fitbg = 0, setbg = 0.2, ellipt_circ = 1)
CLR16Jan = []
analyseSTEDdir(CLR16Jan, options, wdir, files, 
               Ggate = 29, Ygate = 29, 
               DTwoIstar = 0.03, garbageBrightness = 20, junkIstar = 0.40,
               outname = 'CLR16Jan.spots')

analysing image no. 0
b'creating new directory:N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header'
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00287604_spot_0_x_188.0_y_62.0.txt'
number of records is 100260
analysing image no. 1
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00289804_spot_0_x_-84.0_y_-155.0.txt'
number of records is 103142
analysing image no. 2
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00289804_spot_1_x_-86.0_y_-154.0.txt'
number of records is 96672
analysing image no. 3
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00292004_spot_0_x_-148.0_y_-57.0.txt'
number of records is 113524
ROI touches image borders, skipping
analysing image no. 4
header dir already exists
b'N:\\Singlem\\s

number of records is 104665
analysing image no. 39
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00296404_spot_29_x_151.0_y_79.0.txt'
number of records is 101735
analysing image no. 40
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00296404_spot_30_x_66.0_y_108.0.txt'
number of records is 93435
analysing image no. 41
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00296404_spot_31_x_67.0_y_111.0.txt'
number of records is 92454
ROI touches image borders, skipping
analysing image no. 42
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00296404_spot_32_x_71.0_y_111.0.txt'
number of records is 90671
ROI touches image borders, skipping
analysing image no. 43
header dir already exists
b'N:\\Singlem\\singlem

analysing image no. 78
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00303004_spot_9_x_-6.0_y_170.0.txt'
number of records is 100155
analysing image no. 79
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00303004_spot_10_x_19.0_y_-111.0.txt'
number of records is 96231
analysing image no. 80
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00303004_spot_11_x_128.0_y_-72.0.txt'
number of records is 96659
analysing image no. 81
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00303004_spot_12_x_130.0_y_-70.0.txt'
number of records is 91811
analysing image no. 82
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00303004_spot_13_x_-123.0_

analysing image no. 118
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00305204_spot_15_x_25.0_y_-112.0.txt'
number of records is 107201
analysing image no. 119
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00305204_spot_16_x_25.0_y_-111.0.txt'
number of records is 108113
analysing image no. 120
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00305204_spot_17_x_22.0_y_-110.0.txt'
number of records is 110686
ROI touches image borders, skipping
analysing image no. 121
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00305204_spot_18_x_-188.0_y_-95.0.txt'
number of records is 108542
ROI touches image borders, skipping
analysing image no. 122
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\

analysing image no. 156
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00309604_spot_6_x_-10.0_y_143.0.txt'
number of records is 92876
analysing image no. 157
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00309604_spot_7_x_-7.0_y_143.0.txt'
number of records is 95227
analysing image no. 158
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00309604_spot_8_x_-10.0_y_146.0.txt'
number of records is 100046
analysing image no. 159
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00309604_spot_9_x_-109.0_y_161.0.txt'
number of records is 110595
analysing image no. 160
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.00309604_spot_10_x_-49

...

analysing image no. 934
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0029860399999999997_spot_4_x_-80.0_y_127.0.txt'
number of records is 93957
analysing image no. 935
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0029860399999999997_spot_5_x_-82.0_y_128.0.txt'
number of records is 103909
ROI touches image borders, skipping
analysing image no. 936
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0029860399999999997_spot_6_x_7.0_y_130.0.txt'
number of records is 93453
analysing image no. 937
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0029860399999999997_spot_7_x_-81.0_y_131.0.txt'
number of records is 89123
analysing image no. 938
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16

analysing image no. 971
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0036240400000000002_spot_4_x_130.0_y_40.0.txt'
number of records is 95343
analysing image no. 972
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0036240400000000002_spot_5_x_-76.0_y_91.0.txt'
number of records is 101116
analysing image no. 973
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0036240400000000002_spot_6_x_-143.0_y_107.0.txt'
number of records is 89725
ROI touches image borders, skipping
analysing image no. 974
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0036240400000000002_spot_7_x_-62.0_y_135.0.txt'
number of records is 89358
ROI touches image borders, skipping
analysing image no. 975
header dir already exists
b'

analysing image no. 1008
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0037120400000000002_spot_19_x_103.0_y_-63.0.txt'
number of records is 90695
analysing image no. 1009
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0037120400000000002_spot_20_x_102.0_y_-60.0.txt'
number of records is 89319
ROI touches image borders, skipping
analysing image no. 1010
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0037120400000000002_spot_21_x_76.0_y_-43.0.txt'
number of records is 100817
analysing image no. 1011
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\header\\allOverview_Pos_y-0.0037120400000000002_spot_22_x_-47.0_y_10.0.txt'
number of records is 99546
analysing image no. 1012
header dir already exists
b'N:\\Singlem\\singlem20-1\\Ja

[{'filepath': 'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\allOverview_Pos_y-0.00287604_spot_0_x_188.0_y_62.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c9a3a708>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c9a6a3c8>},
 {'filepath': 'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\allOverview_Pos_y-0.00289804_spot_0_x_-84.0_y_-155.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c996aa88>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c9cdcf88>},
 {'filepath': 'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\allOverview_Pos_y-0.00289804_spot_1_x_-86.0_y_-154.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c9f228c8>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c9a65b08>},
 {'filepath': 'N:\\Singlem\\singlem20-1\\January\\16_CLR_overnight\\run1\\all\\allOverview_Pos_y-0.00292004_spot_1_x_-157.0_y_-50.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c989ca48>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c9

<Figure size 432x288 with 0 Axes>

### Januari 21, 2020

In [36]:
path = os.path.join(r'N:\Singlem\singlem20-1\January\21_CLR_origami_NV\run1', '')
savedir = os.path.join(path, 'all')
mergePTUfiles(path, savedir)

Overview_-0.00282294_numberSPOTS_0
Overview_-0.00284494_numberSPOTS_83
Overview_-0.0028669399999999997_numberSPOTS_66
Overview_-0.00288894_numberSPOTS_58
Overview_-0.00291094_numberSPOTS_33
Overview_-0.0029329399999999998_numberSPOTS_27
Overview_-0.0029549399999999997_numberSPOTS_25
Overview_-0.00297694_numberSPOTS_15
Overview_-0.00299894_numberSPOTS_17
Overview_image.tiff


NotADirectoryError: [WinError 267] The directory name is invalid: 'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\Overview_image.tiff/'

In [8]:
wdir = r'N:\Singlem\singlem20-1\January\21_CLR_origami_NV\run1\all'
files = os.listdir(wdir)
options = optionsCluster(fitbg = 0, setbg = 0.2, ellipt_circ = 1)
CLR21Jan = []
analyseSTEDdir(CLR21Jan, options, wdir, files, 
               Ggate = 29, Ygate = 29, 
               DTwoIstar = 0.03, garbageBrightness = 20, junkIstar = 0.40,
               outname = 'CLR21Jan.spots')

analysing image no. 0
b'creating new directory:N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header'
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_0_x_149.0_y_-144.0.txt'
number of records is 97805
analysing image no. 1
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_1_x_71.0_y_-127.0.txt'
number of records is 101922
analysing image no. 2
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_2_x_95.0_y_-85.0.txt'
number of records is 93780
ROI touches image borders, skipping
analysing image no. 3
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_3_x_-61.0_y_-53.0.txt'
number of records is 93947
analysing image no. 4
header dir already exists
b'N:\\Singlem\\

number of records is 88785
analysing image no. 37
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_37_x_-115.0_y_1.0.txt'
number of records is 88758
analysing image no. 38
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_38_x_-113.0_y_1.0.txt'
number of records is 93177
analysing image no. 39
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_39_x_51.0_y_1.0.txt'
number of records is 89640
ROI touches image borders, skipping
analysing image no. 40
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_40_x_46.0_y_4.0.txt'
number of records is 90628
ROI touches image borders, skipping
analysing image no. 41
header dir already exists
b'N:\\Singlem\\singlem20

number of records is 84105
ROI touches image borders, skipping
analysing image no. 74
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_74_x_-75.0_y_140.0.txt'
number of records is 83936
ROI touches image borders, skipping
analysing image no. 75
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_75_x_-74.0_y_140.0.txt'
number of records is 83622
ROI touches image borders, skipping
analysing image no. 76
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_76_x_19.0_y_142.0.txt'
number of records is 84402
ROI touches image borders, skipping
analysing image no. 77
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00284494_spot_77_x_19.0_y_143.0.txt'
number of records is 8

analysing image no. 111
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00288894_spot_28_x_-173.0_y_-39.0.txt'
number of records is 89171
analysing image no. 112
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00288894_spot_29_x_-171.0_y_-39.0.txt'
number of records is 88621
ROI touches image borders, skipping
analysing image no. 113
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00288894_spot_30_x_-182.0_y_-24.0.txt'
number of records is 86720
analysing image no. 114
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00288894_spot_31_x_-184.0_y_-22.0.txt'
number of records is 98591
analysing image no. 115
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\hea

analysing image no. 149
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00291094_spot_8_x_130.0_y_155.0.txt'
number of records is 93940
analysing image no. 150
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00291094_spot_9_x_-27.0_y_159.0.txt'
number of records is 89255
analysing image no. 151
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00291094_spot_10_x_97.0_y_-89.0.txt'
number of records is 88493
analysing image no. 152
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00291094_spot_11_x_100.0_y_-87.0.txt'
number of records is 106097
analysing image no. 153
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.00291094_spot_12_

...

number of records is 99128
analysing image no. 227
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_21_x_39.0_y_-146.0.txt'
number of records is 94605
analysing image no. 228
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_22_x_37.0_y_-145.0.txt'
number of records is 95059
analysing image no. 229
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_23_x_39.0_y_-144.0.txt'
number of records is 93683
analysing image no. 230
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_24_x_-164.0_y_-140.0.txt'
number of records is 94204
analysing image no. 231
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\

number of records is 85857
ROI touches image borders, skipping
analysing image no. 265
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_59_x_-89.0_y_74.0.txt'
number of records is 91560
analysing image no. 266
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_60_x_-49.0_y_76.0.txt'
number of records is 99216
analysing image no. 267
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_61_x_-47.0_y_77.0.txt'
number of records is 96321
analysing image no. 268
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0028669399999999997_spot_62_x_140.0_y_113.0.txt'
number of records is 91275
analysing image no. 269
header dir already exists
b'N:\\

number of records is 87608
analysing image no. 303
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0029549399999999997_spot_4_x_95.0_y_138.0.txt'
number of records is 98910
analysing image no. 304
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0029549399999999997_spot_5_x_95.0_y_140.0.txt'
number of records is 92869
analysing image no. 305
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0029549399999999997_spot_6_x_185.0_y_141.0.txt'
number of records is 94360
analysing image no. 306
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\header\\allOverview_Pos_y-0.0029549399999999997_spot_7_x_182.0_y_144.0.txt'
number of records is 95296
analysing image no. 307
header dir already exists
b'N:\\Singlem\\singlem20-1\\January\\21_CLR_o

[{'filepath': 'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\allOverview_Pos_y-0.00284494_spot_0_x_149.0_y_-144.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c97bebc8>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222cc196708>},
 {'filepath': 'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\allOverview_Pos_y-0.00284494_spot_1_x_71.0_y_-127.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c9d2c808>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222c9c3c348>},
 {'filepath': 'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\allOverview_Pos_y-0.00284494_spot_3_x_-61.0_y_-53.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c9a65748>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222cc600508>},
 {'filepath': 'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\allOverview_Pos_y-0.00284494_spot_4_x_-114.0_y_3.0.ptu',
  'G': <GaussAnalysisPipeline.Channel at 0x222c9b58fc8>,
  'Y': <GaussAnalysisPipeline.Channel at 0x222

<Figure size 432x288 with 0 Axes>

In [10]:
CLR21Jan[0]['filepath']

'N:\\Singlem\\singlem20-1\\January\\21_CLR_origami_NV\\run1\\all\\allOverview_Pos_y-0.00284494_spot_0_x_149.0_y_-144.0.ptu'